# 4) Gender in the Text: Pronouns & Nearby Verbs

**Goal:** Compare relative frequency of pronouns and the verbs near them.

# Setup: Load Texts

This notebook needs **Alice in Wonderland** and **Through the Looking-Glass** as input texts.

**How to provide the texts:**
1. Download books from Project Gutenberg (IDs 11 and 12) as txts. [go to https://www.gutenberg.org/ebooks/11 and https://www.gutenberg.org/ebooks/12]

2. Place two text files in the "data" folder with names:
   - `Wondeland.txt`  (Alice's Adventures in Wonderland)
   - `Looking-Glass.txt` (Through the Looking-Glass)

In [ ]:
import re
from pathlib import Path

In [ ]:

def load_texts(local_alice: str = '../data/Wonderland.txt',
               local_glass: str = '../data/Looking-Glass.txt'):
    """Load Wonderland and Looking-Glass texts from disk.

    Parameters
    ----------
    local_alice : str
        Path to Wonderland text file. Defaults to '../data/Wonderland.txt'.
    local_glass : str
        Path to Looking-Glass text file. Defaults to '../data/Looking-Glass.txt'.

    Returns
    -------
    tuple[str, str]
        (wonderland_text, lookingglass_text).

    Raises
    ------
    FileNotFoundError
        If either file is missing.

    Extra Notes
    -----------
    - Using UTF-8 with `errors='ignore'` avoids codec exceptions on
      older Project Gutenberg dumps or inconsistent encodings.
    """
    p1, p2 = Path(local_alice), Path(local_glass)

    # Fail fast with a clear message if a file is missing
    if not p1.exists():
        raise FileNotFoundError(
            f"Missing file: {p1}\n"
            "→ Please place 'Wonderland.txt' at this path or update load_texts(...)."
        )
    if not p2.exists():
        raise FileNotFoundError(
            f"Missing file: {p2}\n"
            "→ Please place 'Looking-Glass.txt' at this path or update load_texts(...)."
        )

    # Read the files (UTF-8; ignore undecodable bytes to stay robust)
    wonderland   = p1.read_text(encoding='utf-8', errors='ignore')
    lookingglass = p2.read_text(encoding='utf-8', errors='ignore')
    return wonderland, lookingglass

def normalize(text: str) -> str:
    """Normalize a Gutenberg-like text for tokenization.

    Steps
    -----
    1) Heuristically strip Project Gutenberg headers/footers if present
       (looks for *** START ... *** END markers).
    2) Normalize newlines to '\n'.

    Parameters
    ----------
    text : str
        Raw text as loaded from disk (can be empty).

    Returns
    -------
    str
        Cleaned text suitable for tokenization and counting.
    """
    if not text:
        return ''
    # Clip to the main body if markers are present.
    start = text.find('*** START')
    end   = text.find('*** END')
    if start != -1 and end != -1 and end > start:
        text = text[start:end]
    # Normalize Windows line endings.
    return text.replace('\r\n', '\n')

# Load raw texts (forgiving: returns '' if a file is missing)
wonderland_raw, lookingglass_raw = load_texts()

# Normalize for tokenization
wonderland   = normalize(wonderland_raw)
lookingglass = normalize(lookingglass_raw)

print(f"Wonderland chars: {len(wonderland):,} | Looking-Glass chars: {len(lookingglass):,}")


### Helpers: Tokenization

In [ ]:
WORD_RE = re.compile(r"[A-Za-z']+")  # keep apostrophes in words (e.g., don't -> don't)

def words(text: str):
    """Simple word tokenizer (lowercased, ASCII letters + apostrophes).

    Pros
    ----
    - Very fast and dependency-free.
    - Good enough for frequency/keyness demonstrations.

    Cons
    ----
    - No punctuation words, no sentence boundaries, no POS tags.
    - May treat possessives inconsistently across sources.

    Returns
    -------
    list[str]
        Lowercased word words.
    """
    return WORD_RE.findall(text.lower())


def sentences(text: str):
    """Naive sentence splitter using punctuation boundaries.

    Uses a regex to split on '.', '!', '?' followed by whitespace.
    Because this is heuristic, treat results as approximate.

    Returns
    -------
    list[str]
        Sentence-like strings.
    """
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]



wonderland_words = words(wonderland)
lookingglass_words = words(lookingglass)

wonderland_sentences = sentences(wonderland)
lookingglass_sentences = sentences(lookingglass)

print(f"Wonderland words: {len(wonderland_words):,} | Looking-Glass words: {len(lookingglass_words):,}")
print(f"Wonderland sentences: {len(wonderland_sentences):,} | Looking Glass sentences: {len(lookingglass_sentences):,}")


### Pronoun Balance

In [ ]:

from collections import Counter
def pronoun_counts(tokens):
    target = {'he','she','him','her'}
    c = Counter(w for w in tokens if w in target)
    total = sum(c.values())
    return c, total

a_c, a_tot = pronoun_counts(wonderland_words)
g_c, g_tot = pronoun_counts(lookingglass_words)
print("Wonderland:", dict(a_c), "total:", a_tot)
print("LookingGlass:", dict(g_c), "total:", g_tot)


### Verbs Near Pronouns (very naive)

In [ ]:

def verb_like(word):
    # crude heuristic: words ending in common verb suffixes or base forms
    return bool(re.match(r".*(ed|ing|s)$", word)) or word in {"say","says","said","go","goes","went","come","comes","came","think","thinks","thought","see","sees","saw","know","knows","knew"}

def verbs_near_pronouns(tokens, window=2):
    verbs_for = {'he':[], 'she':[]}
    for i,w in enumerate(tokens):
        if w in ('he','she'):
            for j in range(max(0,i-window), min(len(tokens), i+window+1)):
                if j==i: continue
                if verb_like(tokens[j]):
                    verbs_for[w].append(tokens[j])
    return {k: Counter(v).most_common(20) for k,v in verbs_for.items()}

print("Wondeland:", verbs_near_pronouns(wonderland_words) )
print("LookingGlass:", verbs_near_pronouns(lookingglass_words) )

**Prompt:** How do these crude patterns line up with character agency and narrative voice? What errors do you notice, and how would POS (Part OF Speech) tagging improve this?